<a href="https://colab.research.google.com/github/janimiyarj/BDA594-Jani-Miya-Shaik/blob/main/EDM_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.1 MB/s eta 0:00:00
  Created wheel for micawber: filename=micawber-0.5.5-py3-none-any.whl size=22434 sha256=3cf30c39938fe33f473af5986a25c1e0b76d07249dd9fa6b54cd2c4671743e54
  Stored in directory: /root/.cache/pip/wheels/31/3a/1f/bbd88c767423821f2fe497c0ada47581292f96419b56d689c0
Successfully built micawber
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not in

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp
import pymysql
import warnings
import pymysql
import sys
import os

warnings.filterwarnings('ignore')

In [ ]:
host = 'database-1.c1r9lsqj9mxc.us-east-1.rds.amazonaws.com'
user = 'admin'
password = 'Arbaz786$'
database = 'gdb041'

connection = pymysql.connect(host=host, user=user, passwd=password, database=database)
with connection:
    cur = connection.cursor()
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print("Database version: {} ".format(version[0]))

Database version: 8.0.33 


In [ ]:
def run_query(q: str) -> pd.DataFrame:
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        return pd.read_sql(q, conn)


def run_command(c: str):
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        conn.isolation_level = None
        conn.execute(c)

In [ ]:
def show_tables() -> pd.DataFrame:
    q = """
          SELECT table_name
          FROM information_schema.tables
          WHERE table_schema = 'gdb041' AND table_type = 'BASE TABLE';
        """
    return run_query(q)


def get_table_row_count(tablename) -> int:
    q = (
        """
        SELECT
            COUNT(1)
        FROM %s;
        """
        % tablename
    )
    return run_query(q)["COUNT(1)"][0]


tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables['TABLE_NAME']]

tables

,TABLE_NAME,row_count
0,dim_customer,209
1,dim_market,27
2,dim_product,397
3,fact_forecast_monthly,1885941
4,fact_sales_monthly,1425706
5,freight_cost,135
6,gross_price,1197
7,manufacturing_cost,1197
8,post_invoice_deductions,2063076
9,pre_invoice_deductions,1045


# Building a dashboard

Bar Chart - Total Sales by Product Category:

In [ ]:
import datapane as dp

Total_Sales_by_Product_Category_query = """
SELECT category,
       SUM(sold_quantity) AS total_sold_quantity,
       SUM(sold_quantity * gross_price) AS total_revenue
FROM gdb041.fact_sales_monthly F
JOIN gdb041.gross_price G ON F.product_code = G.product_code
AND YEAR(F.date) = G.fiscal_year
GROUP BY category;
"""

total_sales_df = run_query(Total_Sales_by_Product_Category_query)

dp.DataTable(total_sales_df)


In [ ]:
total_sales_plot = px.bar(total_sales_df, x='category', y='total_revenue', color = 'category', title='Total Sales by Product Category')
total_sales_plot.show()

Line Chart - Monthly Sales Trends for a Specific Product:

In [ ]:
Monthly_Sales_Trends_for_a_Specific_Product_query="""
SELECT date, SUM(sold_quantity) AS monthly_sales
FROM fact_sales_monthly
WHERE product_code = 'A0418150101'
GROUP BY date
ORDER BY date;
"""
sales_trends_df=run_query(Monthly_Sales_Trends_for_a_Specific_Product_query)
dp.DataTable(sales_trends_df)

In [ ]:
monthly_tales_trends = px.line(sales_trends_df, x='date', y='monthly_sales', title='Monthly Sales Trends for a Specific Product')
monthly_tales_trends.show()

Pie Chart - Market Share of Top Products:

In [ ]:
Market_Share_of_Top_Products_query="""
WITH top_products AS (
  SELECT product_code, SUM(sold_quantity) AS total_sold
  FROM fact_sales_monthly
  GROUP BY product_code
  ORDER BY total_sold DESC
  LIMIT 5
)
SELECT tp.product_code, tp.total_sold, SUM(total_sold) OVER () AS total_sales
FROM top_products tp;
"""
market_share_df=run_query(Market_Share_of_Top_Products_query)
dp.DataTable(market_share_df)

In [ ]:
market_share = px.pie(market_share_df, names='product_code', values='total_sold', title='Market Share of Top Products')
market_share.show()

Pie chart - Top 3 Customers by Purchases:

In [ ]:
Top_3_Customers_by_Purchases_query="""
SELECT customer_code, customer_name, SUM(sold_quantity) AS total_purchases
FROM gdb041.fact_sales_monthly
WHERE YEAR(date) = YEAR('2020-01-01 00:00:00')
GROUP BY customer_code, customer_name
ORDER BY total_purchases DESC
LIMIT 3;
"""

top_purchases_df=run_query(Top_3_Customers_by_Purchases_query)

dp.DataTable(top_purchases_df)

In [ ]:
top_3_purchases = px.pie(top_purchases_df, names='customer_name', values='total_purchases', title='Top 3 Customers by Purchases')
top_3_purchases.show()

Time Series - Sales Trends Over Time:

In [ ]:
Sales_Trends_Over_Time_query="""
SELECT date, SUM(sold_quantity) AS total_quantity_sold
FROM fact_sales_monthly
GROUP BY date
ORDER BY date;
"""

sales_df=run_query(Sales_Trends_Over_Time_query)

dp.DataTable(sales_df)

In [ ]:
sales_trend = px.line(sales_df, x='date', y='total_quantity_sold', title='Sales Trends Over Time')
sales_trend.show()

Bar Plot - Total Number of Products sold by Product Category:

In [ ]:
Total_Number_of_Products_sold_by_Product_Category_query="""
SELECT category,
       SUM(sold_quantity) AS total_sold_quantity,
       SUM(sold_quantity * gross_price) AS total_revenue
FROM gdb041.fact_sales_monthly F
JOIN gdb041.gross_price G ON F.product_code = G.product_code
AND YEAR(F.date) = G.fiscal_year
GROUP BY category;
"""

total_df=run_query(Total_Number_of_Products_sold_by_Product_Category_query)
dp.DataTable(total_df)

In [ ]:
total_sold_plot = px.bar(total_df, x='category', y='total_sold_quantity', color = 'category', title='Distribution of Manufacturing Costs by Product')
total_sold_plot.show()

In [ ]:
import altair as alt
import datapane as dp
dp.enable_logging()

Putting it into a Datapane report

In [ ]:
customers = tables[tables["TABLE_NAME"] == "dim_customer"]["row_count"].values[0]
markets = tables[tables["TABLE_NAME"] == "dim_market"]["row_count"].values[0]
products = tables[tables["TABLE_NAME"] == "dim_product"]["row_count"].values[0]
post_invoice_deductions = tables[tables["TABLE_NAME"] == "post_invoice_deductions"]["row_count"].values[0]

In [ ]:
r = dp.View(
    dp.Group(
        dp.BigNumber(heading="Total customers", value=customers),
        dp.BigNumber(heading="Total markets", value=markets),
        dp.BigNumber(heading="Total products sold", value=products),
        columns=3,
        name="Little_group",
    ),
    dp.Plot(total_sales_plot, name="total_sales"),
    dp.Plot(monthly_tales_trends, name="monthly_tales_trends"),
    dp.Group(dp.Plot(market_share), dp.Plot(sales_trend), dp.Plot(top_3_purchases), dp.Plot(total_sold_plot), columns=2),
)

dp.save_report(r, "report.html", open=True)

[01:08:00] [DEBUG] <View version="1" fragment="false">
  <Group name="Little_group" columns="3" valign="top">
    <BigNumber heading="Total customers" value="209" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total markets" value="27" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total products sold" value="397" is_positive_intent="false" is_upward_change="false"/>
  </Group>
  <Plot type="application/vnd.plotly.v1+json" name="total_sales" responsive="true" scale="1.0" src="ref://327e8c233a"/>
  <Plot type="application/vnd.plotly.v1+json" name="monthly_tales_trends" responsive="true" scale="1.0" src="ref://a7e457a162"/>
  <Group columns="2" valign="top">
    <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://e90514c238"/>
    <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://09ebbb4dda"/>
    <Plot type="application/vnd.plotly.v1+json" responsive="true" s

App saved to ./report.html

In [ ]:
import IPython
IPython.display.HTML(filename='report.html')

In [ ]:
from google.colab import files

# Download the HTML file
files.download('report.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>